### Import Modules

In [48]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import core.functions_warcpy as fwa
import core.functions as fun

### Initialize Variables

In [2]:
from core.setvars import *

extendedTrans = os.path.join(home, 'fiis_trans')
extTrans_tidy = os.path.join(home, 'tidyTrans')

inletLines = os.path.join(home, 'inletLines')
ShorelinePts = os.path.join(home, 'SLpts_utm')
dlPts = os.path.join(home, 'DLpts_utm')
dhPts = os.path.join(home, 'DHpts_utm')
armorLines = os.path.join(home, 'armorLines')
elevGrid = os.path.join(home, 'DEM')
elevGrid_5m = os.path.join(home, 'DEM_5m')
barrierBoundary = os.path.join(home, 'bndpoly_2sl')  
shoreline = os.path.join(home, 'ShoreBetweenInlets')

SubType = os.path.join(home, 'FI12_SubType')
VegType = os.path.join(home, 'FI12_VegType')
VegDens = os.path.join(home, 'FI12_VegDen')
GeoSet = os.path.join(home, 'FI12_GeoSet')

site: FireIsland
year: 2012
setvars.py initialized variables.


### Merge from excel file

In [3]:
# 1. Import data from xls 
# File name
xls_path = r"\\IGSAGIEGGS-CSGG\Thieler_Group\Commons_DeepDive\DeepDive\NewYork\AnthropogenicData"
xls_name = "FI2012_trans_5mPtsUpCombinedsubFixALL2.xlsx"

xltbl = pd.read_excel(os.path.join(xls_path, xls_name))

In [4]:
xltbl.head()

,OBJECTID_1,OBJECTID,BASELINEID,TRANSORDER,PROCTIME,AUTOGEN,ENDX,ENDY,AZIMUTH,SHAPE_LENG,...,Dist_MHWbay,SplitSort,DistSegDH,DistSegDL,DistSegArm,PointZ,PointSlp,Development,Nourishment,Construction
0,1,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,1,-99999.0,-99999.0,-99999.0,0.91820,9.842591,111,111,111
1,2,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,2,-99999.0,-99999.0,-99999.0,1.19412,7.487503,111,111,111
2,3,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,3,-99999.0,-99999.0,-99999.0,0.85836,5.800764,111,111,111
3,4,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,4,-99999.0,-99999.0,-99999.0,1.05932,6.581713,111,111,111
4,5,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,5,-99999.0,-99999.0,-99999.0,0.88016,10.083154,111,111,111


In [5]:
xltbl.columns

Index(['OBJECTID_1', 'OBJECTID', 'BASELINEID', 'TRANSORDER', 'PROCTIME',
       'AUTOGEN', 'ENDX', 'ENDY', 'AZIMUTH', 'SHAPE_LENG', 'TRANSECTID', 'LRR',
       'LR2', 'LSE', 'LCI90', 'TID2', 'LRR2', 'LR22', 'LSE2', 'LCI902',
       'SL_Lon', 'SL_Lat', 'SL_easting', 'SL_northing', 'Bslope', 'Arm_Lon',
       'Arm_Lat', 'Arm_easting', 'Arm_northing', 'Arm_z', 'DH_Lon', 'DH_Lat',
       'DH_easting', 'DH_northing', 'DH_z', 'DL_Lon', 'DL_Lat', 'DL_easting',
       'DL_northing', 'DL_z', 'DL_zMHW', 'DH_zMHW', 'Arm_zMHW', 'DistDH',
       'DistDL', 'DistArm', 'Dist2Inlet', 'WidthLand', 'WidthFull',
       'WidthPart', 'Source_beachwidth', 'MLW_easting', 'MLW_northing',
       'beach_h_MLW', 'beachWidth_MLW', 'ORIG_OID', 'seg_y', 'seg_x',
       'Dist_Seg', 'Dist_MHWbay', 'SplitSort', 'DistSegDH', 'DistSegDL',
       'DistSegArm', 'PointZ', 'PointSlp', 'Development', 'Nourishment',
       'Construction'],
      dtype='object')

## Join anthro data to transects

1. Convert xls spreadsheet to points 
2. Select the first points along each transects and create new FC
3. Spatial Join the new FC to the updated transects 
    - one to one
    - keep all target features
    - keep only the ID fields and the three anthro fields (and the transect fields [LRR, etc.]?)
    - intersect

4. Join the transect values to the pts based on sort_ID


In [6]:
tr_w_anthro = os.path.join(arcpy.env.workspace, 'fiis_trans_wAnthro')

In [8]:
trans_df2 = fwa.FCtoDF(tr_w_anthro, id_fld=tID_fld, dffields=['Development', 'Nourishment','Construction'])

Converting feature class to array...
Converting array to dataframe...


In [12]:
# Get existing DFs
pts_df = pd.read_pickle(os.path.join(scratch_dir, transPts_null+'.pkl'))
trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'.pkl'))

# Join anthro fields to trans and points DFs
trans_df = fun.join_columns(trans_df, trans_df2) 
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)

# Save dataframes to open elsewhere or later
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'_anthro.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, transPts_null+'_anthro.pkl'))

,SplitSort,GeoSet,seg_x,seg_y,SubType,VegDens,VegType,ptSlp,ptZ,sort_ID,...,ub_feat,Dist2Inlet,WidthFull,WidthLand,WidthPart,mean_Zmhw,max_Zmhw,Construction,Development,Nourishment
0,0,1.0,642131.5400,4.498927e+06,4444.0,111.0,11.0,4.445196,0.68200,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
1,1,1.0,642133.1304,4.498932e+06,4444.0,111.0,11.0,4.445196,0.68200,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
2,2,1.0,642134.7209,4.498937e+06,4444.0,111.0,11.0,7.459952,0.39028,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
3,3,2.0,642115.6352,4.498880e+06,7777.0,666.0,77.0,10.083154,0.88016,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
4,4,1.0,642117.2257,4.498885e+06,4444.0,111.0,11.0,10.088981,0.86376,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0


In [22]:
# Recode the values for CSV
pts_df4csv = pts_df.replace({'SubType': {7777:'{1111, 2222}', 1000:'{1111, 3333}'}, 
                              'VegType': {77:'{11, 22}', 88:'{22, 33}', 99:'{33, 44}'},
                              'VegDens': {666: '{111, 222}', 777: '{222, 333}', 
                                          888: '{333, 444}', 999: '{222, 333, 444}'}})

pts_df4csv.to_pickle(os.path.join(scratch_dir, transPts_null+'_4csv.pkl'))

pts_df4csv.sample(5)

,SplitSort,seg_x,seg_y,Dist_Seg,Dist_MHWbay,DistSegDH,DistSegDL,DistSegArm,ptZ,ptSlp,...,WidthLand,WidthFull,uBW,uBH,ub_feat,mean_Zmhw,max_Zmhw,Construction,Development,Nourishment
66615,66615,677018.1595,4.509975e+06,230.000032,82.220969,143.009575,149.259519,NaN,0.60388,2.552202,...,312.221001,312.221001,80.787403,3.090523,DL,0.957232,4.469680,111.0,111.0,222.0
85619,85619,687214.1370,4.514490e+06,345.696325,-131.024561,229.025318,240.263141,NaN,0.35092,0.961539,...,749.924166,855.620515,105.471719,4.944896,DL,0.591762,7.122921,111.0,111.0,222.0
66511,66511,677044.7910,4.509796e+06,55.000040,286.701757,32.017383,25.841729,NaN,1.81084,2.832443,...,341.701797,341.701797,80.572297,2.878211,DL,0.657066,3.999680,111.0,111.0,222.0
31938,31938,657490.3763,4.501733e+06,434.999996,137.250023,NaN,NaN,NaN,0.76025,NaN,...,572.250019,572.250019,NaN,NaN,NaN,NaN,3.859720,111.0,333.0,222.0
87919,87919,688190.4957,4.514639e+06,270.000020,275.964937,179.786747,188.534853,NaN,5.43132,6.142564,...,545.964957,545.964957,81.470031,3.533009,DL,1.878405,5.923080,111.0,111.0,222.0


In [15]:
# pID_fld needs to be among the columns
if not pID_fld in pts_df4csv.columns:
    pts_df4csv.reset_index(drop=False, inplace=True)

# Save CSV in scratch_dir
csv_fname = os.path.join(scratch_dir, transPts_fill +'.csv')
pts_df4csv.to_csv(os.path.join(scratch_dir, transPts_fill +'.csv'), na_rep=fill, index=False)
print("OUTPUT: {}".format(csv_fname))

try:
    xls_fname = os.path.join(scratch_dir, transPts_fill +'.xlsx')
    pts_df4csv.to_excel(xls_fname, na_rep=fill, index=False)
    print("OUTPUT: {}".format(xls_fname))
except:
    print("No Excel file created. You'll have to do it yourself from the CSV.")

OUTPUT: \\Mac\stor\Projects\TransectExtraction\FireIsland2012\scratch\FireIsland2012_transPts_fill.csv
No Excel file created. You'll have to do it yourself from the CSV.


In [18]:
tr_w_LRR = os.path.join(arcpy.env.workspace, 'fiis_trans_wLRR')
trans_df3 = fwa.FCtoDF(tr_w_LRR, id_fld=tID_fld, dffields=['LRR'])

# Load dataframes
trans_df= pd.read_pickle(os.path.join(scratch_dir, '20180108', extTrans_null+'_anthro.pkl'))
pts_df= pd.read_pickle(os.path.join(scratch_dir, '20180108', transPts_null+'_anthro.pkl'))

# Join anthro fields to trans and points DFs
trans_df = fun.join_columns(trans_df, trans_df3) 
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)

# Save dataframes to open elsewhere or later
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'_anthroLRR.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, transPts_null+'_anthroLRR.pkl'))

Converting feature class to array...
Converting array to dataframe...


In [21]:
# Sort columns
pts_df = pts_df.reindex_axis(sorted_pt_flds, axis=1)

# Recode
pts_df4csv = pts_df.replace({'GeoSet': {9999:-99999},
                             'SubType': {7777:'{1111, 2222}', 1000:'{1111, 3333}', 9999:-99999}, 
                              'VegType': {77:'{11, 22}', 88:'{22, 33}', 99:'{33, 44}', 9999:-99999},
                              'VegDens': {666: '{111, 222}', 777: '{222, 333}', 
                                          888: '{333, 444}', 999: '{222, 333, 444}', 9999:-99999}})

# Save as pickle
pts_df4csv.to_pickle(os.path.join(scratch_dir, transPts_null+'_4csv.pkl'))

# Save as CSV
csv_fname = os.path.join(scratch_dir, transPts_fill +'.csv')
pts_df4csv.to_csv(os.path.join(scratch_dir, transPts_fill +'.csv'), na_rep=fill, index=False)
print("OUTPUT: {}".format(csv_fname))


OUTPUT: \\Mac\stor\Projects\TransectExtraction\FireIsland2012\scratch\FireIsland2012_transPts_fill.csv


Fixing measure_Dist2Inlet() to work when shorelineXinlet intersect point falls directly on transect

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import core.functions_warcpy as fwa
import core.functions as fun

from core.setvars import *

armorLines = os.path.join(home, 'BP_armorshoreward_2014')
sl2trans_df = pd.read_pickle(os.path.join(scratch_dir, 'sl2trans.pkl'))

site: Rockaway
year: 2014
Path to project directory (e.g. \\Macolume\dir\FireIsland2014): \\Mac\stor\Projects\TransectExtraction\Rockaway2014
setvars.py initialized variables.


In [9]:
shoreline = os.path.join(home, 'ShoreBetweenInlets')
in_trans = extendedTrans = os.path.join(home, 'extTrans')
inletLines = os.path.join(home, 'BP_inletLines_2014')
tID_fld = 'sort_ID'

# Calc Dist2Inlet in new dataframe 
# dist_df = fwa.measure_Dist2Inlet(shoreline, extendedTrans, inletLines, tID_fld)

utmSR = arcpy.Describe(in_trans).spatialReference
df = pd.DataFrame(columns=[tID_fld, 'Dist2Inlet']) # initialize dataframe
inlets = [row[0] for row in arcpy.da.SearchCursor(inletLines, ("SHAPE@"))]

cursor = arcpy.da.SearchCursor(shoreline, ("SHAPE@"))
row = cursor.next()
line = row[0]
line

<Polyline object at 0xcee2240[0x1052ea58]>

In [47]:
for [transect, tID] in arcpy.da.SearchCursor(in_trans, ("SHAPE@",  tID_fld)):
    if not line.disjoint(transect):
        # 1. cut shoreline at the transect
        [rseg, lseg] = line.cut(transect)
        # 2. if the shoreline segment touches any inlet, get the segment length. 
        #    If it doesn't touch an inlet, length is set to NaN to remove it from consideration.
        # in case of multipart features, use only the first part
        lenR = arcpy.Polyline(rseg.getPart(0), utmSR).length if not all(rseg.disjoint(i) for i in inlets) else np.nan
        lenL = arcpy.Polyline(lseg.getPart(0), utmSR).length if not all(lseg.disjoint(i) for i in inlets) else np.nan
        # 3. If shoreline and transect intersect on an inlet line, return 0 because transect is at an inlet.
        #    Only check for overlap at segments that touch an inlet (not NaN).
        xpt = line.intersect(transect, 1) # point where shoreline and transect intersect
        lenR = 0 if not np.isnan(lenR) and not all(xpt.disjoint(i) for i in inlets) else lenR
        lenL = 0 if not np.isnan(lenL) and not all(xpt.disjoint(i) for i in inlets) else lenL
        mindist = np.nanmin([lenR, lenL])
        df = df.append({tID_fld:tID, 'Dist2Inlet':mindist}, ignore_index=True)
        try:
            dist_prev = pd.to_numeric(df.loc[df[tID_fld]==tID-1, 'Dist2Inlet'])
            if any(abs(dist_prev - mindist) > 300):
                print("CAUTION: Large change in Dist2Inlet values between transects {} ({} m) and {} ({} m).".format(tID-1, dist_prev, tID, mindist))
        except:
            print("Error-catching is not working in Dist2Inlet.")
            pass
df

# Looks good! 

,sort_ID,Dist2Inlet
0,6.0,48.808948
1,7.0,99.424597
2,8.0,149.464110
3,9.0,199.548346
4,10.0,249.753352
5,11.0,299.969340
6,12.0,350.055564
7,13.0,400.106897
8,14.0,450.117768
9,15.0,500.119950


In [45]:
np.isnan(lenR)

True

In [31]:
for [transect, tID] in arcpy.da.SearchCursor(in_trans, ("SHAPE@",  tID_fld)):
    if tID == 348:
        break
print(tID)

348


In [33]:
line.disjoint(transect)
[rseg, lseg] = line.cut(transect) # rseg doesn't exist and length of lseg is 17161
lenR = arcpy.Polyline(rseg.getPart(0), utmSR).length if not all(rseg.disjoint(i) for i in inlets) else np.nan
lenL = arcpy.Polyline(lseg.getPart(0), utmSR).length if not all(lseg.disjoint(i) for i in inlets) else np.nan

print(lenR, lenL)
rseg.disjoint(transect)

17113.137653438618 48.65892948035971


False

In [22]:
xpt = line.intersect(transect, 1) # where shoreline and transect intersect
# is xpt the exact end of the shoreline and/or exactly on an inlet? better if it's on an inlet because the shoreline could end without an inlet
# xpt.disjoint(inl)
if not all(xpt.disjoint(i) for i in inlets):
    print('winner?')

In [ ]:
for inl in inlets:
    if not transect.disjoint(inl):
        xpt = transect.intersect(inl)

## Convert CSV to FC

In [51]:
path = r'\\Mac\Home\Downloads\13CNT05_morphology'
datapre = '13CNT05'
fname = '13CNT05_morphology.csv'
fill = 999
wgs84 = arcpy.SpatialReference(4326)

# import CSV as DF and remove nulls/fills
df = pd.read_csv(os.path.join(path, fname))
df = df[df != fill]

# subset to state 
df = df[df.state == 16]

# subset and convert to FC
dataname = os.path.splitext(fname)[0]+'_DT'
df_dt = df[df.feature_type == 'DT'] 
df_dt.to_csv(os.path.join(path, '{}_ny_dlo.csv'.format(datapre)), index=False)

# DFtoFC throwing error "table name is invalid [13CNT05_morphology_DT]"
fc = fwa.DFtoFC(df_dt, os.path.join(arcpy.env.scratchGDB, dataname), spatial_ref=wgs84, xy=['lon', 'lat'], keep_fields='all', fill=fill)
arcpy.FeatureClassToFeatureClass_conversion(fc, path, dataname+'.shp')

RuntimeError: The table name is invalid. [13CNT05_morphology_DT]

In [ ]:

# subset and convert to FC
dataname = os.path.splitext(fname)[0]+'_DC'
df_dc = df[df.feature_type == 'DC']
df_dc.to_csv(os.path.join(path, '{}_ny_dhi.csv'.format(datapre)), index=False)

fc = fwa.DFtoFC(df_dc, os.path.join(arcpy.env.scratchGDB, dataname), spatial_ref=wgs84, xy=['lon', 'lat'], keep_fields='all', fill=fill)
arcpy.FeatureClassToFeatureClass_conversion(fc, path, dataname+'.shp')

# subset and convert to FC
dataname = os.path.splitext(fname)[0]+'_SL'
df_sl = df[df.feature_type == 'SL'] 
df_sl.to_csv(os.path.join(path, '{}_ny_sl.csv'.format(datapre)), index=False)

fc = fwa.DFtoFC(df_sl, os.path.join(arcpy.env.scratchGDB, dataname), spatial_ref=wgs84, xy=['lon', 'lat'], keep_fields='all', fill=fill)
arcpy.FeatureClassToFeatureClass_conversion(fc, path, dataname+'.shp')